In [ ]:
# import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random 
from scipy.stats import mannwhitneyu

In [ ]:
# setting
plt.rcParams['font.family']= 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['font.size'] = 18
plt.rcParams["figure.dpi"] = 200

In [ ]:
# Supplementary Fig. 8a
file_path = "../previous_data/ChIP_H3K4me123_WT_atxr3.rpkm.tsv"
df_chip = pd.read_csv(file_path,sep="\t",index_col=0)

In [ ]:
# List of H3K4me2 diel oscillating genes (DOGs)
file_path = f"../data/list_of_DOGs/H3K4me2_DOGs.bed"
H3K4me2_DOGs = pd.read_csv(file_path,sep="\t",header=None).iloc[:,3].values

In [ ]:
mask = df_chip.index.isin(H3K4me2_DOGs)
print(mask.sum())
hm_list = ["H3K4me1","H3K4me2","H3K4me3"]

fig,ax = plt.subplots(1,3,figsize=(15,5))
for i,hm in enumerate(hm_list):
    print(hm)
    x = f"{hm}_WT"
    y = f"{hm}_atxr3"

    # plot
    sns.scatterplot(data=df_chip[~mask],x=x,y=y,
                    s=5,alpha=1,c="tab:grey",ax=ax[i])

    sns.scatterplot(data=df_chip[mask],x=x,y=y,
                    s=5,alpha=1,color="red",ax=ax[i])
    
    # format figure
    ax[i].set_xlim(0,np.percentile(df_chip[x],99))
    ax[i].set_ylim(0,np.percentile(df_chip[x],99))
    ax[i].set_xlabel("")
    ax[i].set_ylabel("")
    ax[i].set_aspect('equal', adjustable='box')

In [ ]:
# Supplementary Fig. 8b
random.seed(1)

# H3K4me2 DOGs
mask_dogs = df_chip.index.isin(H3K4me2_DOGs)
N = mask_dogs.sum()
df_chip_dogs = df_chip[mask_dogs]

# H3K4me2 non-DOGs
mask_non_dogs = df_chip.index.isin(random.sample(list(df_chip.index[~mask_dogs]),N))
df_chip_non_dogs = df_chip[mask_non_dogs]

#print(N,df_chip_dogs.index[:3],df_chip_non_dogs.index[:3])

hm_list = ["H3K4me1","H3K4me2","H3K4me3"]
fig, ax = plt.subplots(1,1)
for i in range(3):
    hm = hm_list[i]
    data = {2*i:df_chip_dogs[f"{hm}_atxr3"]-df_chip_dogs[f"{hm}_WT"],
            2*i+1:df_chip_non_dogs[f"{hm}_atxr3"]-df_chip_non_dogs[f"{hm}_WT"],}

    # plot
    sns.boxplot(data,palette=["red","tab:grey"])

    # Mann-Whitney U test
    p = mannwhitneyu(data[2*i], data[2*i+1], alternative="two-sided")[1]
    print(f"{hm} : {p:.2e}")

# format figure
ax.set_ylim(-50,30)
ax.set_xticks([0.5,2.5,4.5])
ax.set_xticklabels(["H3K4me1","H3K4me2","H3K4me3"])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)